## Pandas - Como otimizar o uso de memória

O objetivo desse estudo é aumentar a eficiência do uso da biblioteca Pandas, diminuindo o uso de memória.<br>
Utilizando o ``dtypes`` de forma "mais correta".

In [25]:
import pandas as pd
import numpy as np

from numpy import size

## Dataframe

Criando o dataframe com a biblioteca ``numpy``, vou criar o dataframe inspirado em um time de futebol.

In [26]:
def setup_dataframe(size):
    df = pd.DataFrame()
    df['posicao'] = np.random.choice(['goleiro', 'zagueiro', 'meio campo', 'lateral direito', 'lateral esquerdo', 'atacante'], size)
    df['idade'] = np.random.randint(10, 50, size)
    df['time'] = np.random.choice(['amarelo', 'vermelho', 'azul', 'verde'], size)
    df['vencedor'] = np.random.choice(['sim', 'nao'], size)
    df['probabilidade'] = np.random.uniform(0, 1, size)
    return df

In [27]:
%%time
df = setup_dataframe(1_000_000)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   posicao        1000000 non-null  object 
 1   idade          1000000 non-null  int32  
 2   time           1000000 non-null  object 
 3   vencedor       1000000 non-null  object 
 4   probabilidade  1000000 non-null  float64
dtypes: float64(1), int32(1), object(3)
memory usage: 34.3+ MB
CPU times: total: 1.48 s
Wall time: 1.66 s


In [28]:
df['idade_rank'] = df.groupby(['time', 'posicao'])['idade'].rank()
df['probabilidade_rank'] = df.groupby(['time', 'posicao'])['probabilidade'].rank()
df['vencedor_probabilidade_rank'] = df.groupby(['time', 'posicao', 'vencedor'])['probabilidade'].rank()

In [29]:
%timeit df['idade_rank'] = df.groupby(['time', 'posicao'])['idade'].rank()
%timeit df['probabilidade_rank'] = df.groupby(['time', 'posicao'])['probabilidade'].rank()
%timeit df['vencedor_probabilidade_rank'] = df.groupby(['time', 'posicao', 'vencedor'])['probabilidade'].rank()

1.2 s ± 294 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
The slowest run took 4.03 times longer than the fastest. This could mean that an intermediate result is being cached.
1.14 s ± 685 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
951 ms ± 275 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Alterando os Tipos das variáveis

### Variáveis do tipo ``object``
Podemos observar após a alteração das variáveis do tipo ``object`` para ``category`` o uso de memória diminui consideravelmente.

In [30]:
%%time
df = setup_dataframe(1_000_000)
df['posicao'] = df['posicao'].astype('category')
df['time'] = df['time'].astype('category')
df['vencedor'] = df['vencedor'].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype   
---  ------         --------------    -----   
 0   posicao        1000000 non-null  category
 1   idade          1000000 non-null  int32   
 2   time           1000000 non-null  category
 3   vencedor       1000000 non-null  category
 4   probabilidade  1000000 non-null  float64 
dtypes: category(3), float64(1), int32(1)
memory usage: 14.3 MB
CPU times: total: 1.19 s
Wall time: 1.29 s


### Variáveis do tipo ``int``

Podemos fazer ``downcast`` dos valores ``int``. Isso quer dizer que podemos alterar de ``int64`` para ``int16`` ou ``int8``. <br>
Entretanto, esse processo deve ser feito com cautela, os dados do dataframe podem sofrer alterações os deixando diferentes do origial.

Essa tabela é um "mini guia" para o downcasting.

- 8 bit inicia em –128 até 127
- 16 bit inicia em  –32768 até 32767
- 32 bit inicia em –2147483648 até 2147483647
- 64 bit inicia em -9223372036854775808 até -9223372036854775808

Vou iniciar com a variável ``idade``.<br>
Sabemos que seres humanos podem viver até os 130 anos. As chances são muito baixas até esse idade.<br>
Com isso em mente, vou alterar de ``int32`` para ``int8``.

In [31]:
df['idade'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1000000 entries, 0 to 999999
Series name: idade
Non-Null Count    Dtype
--------------    -----
1000000 non-null  int32
dtypes: int32(1)
memory usage: 3.8 MB


In [32]:
print(df['idade'].max())
print(df['idade'].min())

49
10


In [33]:
df['idade'] = df['idade'].astype('int8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype   
---  ------         --------------    -----   
 0   posicao        1000000 non-null  category
 1   idade          1000000 non-null  int8    
 2   time           1000000 non-null  category
 3   vencedor       1000000 non-null  category
 4   probabilidade  1000000 non-null  float64 
dtypes: category(3), float64(1), int8(1)
memory usage: 11.4 MB


## Donwcast ``floats``

Seguindo a mesma linha do tipo ``int``, podemos alterar o tipo ``float`` e diminuir o uso da memória.<br>
É necessário ter certeza que os dados não serão alterados.

In [34]:
df['probabilidade'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1000000 entries, 0 to 999999
Series name: probabilidade
Non-Null Count    Dtype  
--------------    -----  
1000000 non-null  float64
dtypes: float64(1)
memory usage: 7.6 MB


In [35]:
print(df['probabilidade'].max())
print(df['probabilidade'].min())

0.9999984780402752
4.0518407051770566e-07


In [36]:
df['probabilidade']

0         0.208839
1         0.226900
2         0.552175
3         0.295297
4         0.984637
            ...   
999995    0.397899
999996    0.213759
999997    0.543745
999998    0.133332
999999    0.007853
Name: probabilidade, Length: 1000000, dtype: float64

In [37]:
df['probabilidade'] = df['probabilidade'].astype('float32')
df['probabilidade']

0         0.208839
1         0.226900
2         0.552175
3         0.295297
4         0.984637
            ...   
999995    0.397899
999996    0.213759
999997    0.543745
999998    0.133332
999999    0.007853
Name: probabilidade, Length: 1000000, dtype: float32

In [38]:
df[ 'probabilidade'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1000000 entries, 0 to 999999
Series name: probabilidade
Non-Null Count    Dtype  
--------------    -----  
1000000 non-null  float32
dtypes: float32(1)
memory usage: 3.8 MB


## Variáveis booleanas

Também podemos alterar variáveis "sim/não" para o tipo ``bool``, isso ajudará a diminuir o uso de memória.

In [39]:
df['vencedor'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1000000 entries, 0 to 999999
Series name: vencedor
Non-Null Count    Dtype   
--------------    -----   
1000000 non-null  category
dtypes: category(1)
memory usage: 976.8 KB


In [40]:
df.head()

,posicao,idade,time,vencedor,probabilidade
0,lateral esquerdo,26,amarelo,sim,0.208839
1,zagueiro,45,verde,nao,0.226900
2,lateral direito,47,amarelo,sim,0.552175
3,lateral esquerdo,24,vermelho,sim,0.295297
4,lateral esquerdo,18,verde,nao,0.984637


In [41]:
df['vencedor'].map({'sim': True, 'não': False})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype   
---  ------         --------------    -----   
 0   posicao        1000000 non-null  category
 1   idade          1000000 non-null  int8    
 2   time           1000000 non-null  category
 3   vencedor       1000000 non-null  category
 4   probabilidade  1000000 non-null  float32 
dtypes: category(3), float32(1), int8(1)
memory usage: 7.6 MB


## Prova final

In [42]:
def alterando_dtypes(df):
    df['posicao'] = df['posicao'].astype('category')
    df['time'] = df['time'].astype('category')
    df['vencedor'] = df['vencedor'].astype('category')
    df['probabilidade'] = df['probabilidade'].astype('float32')
    df['vencedor'].map({'sim': True, 'não': False})
    return df

In [43]:
%%time
df = setup_dataframe(1_000_000)
%timeit df['idade_rank'] = df.groupby(['time', 'posicao'])['idade'].rank()
%timeit df['probabilidade_rank'] = df.groupby(['time', 'posicao'])['probabilidade'].rank()
%timeit df['vencedor_probabilidade_rank'] = df.groupby(['time', 'posicao', 'vencedor'])['probabilidade'].rank()

843 ms ± 477 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
866 ms ± 256 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
991 ms ± 224 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
CPU times: total: 17.5 s
Wall time: 22.4 s


In [44]:
%%time
df = setup_dataframe(1_000_000)
df = alterando_dtypes(df)
%timeit df['idade_rank'] = df.groupby(['time', 'posicao'])['idade'].rank()
%timeit df['probabilidade_rank'] = df.groupby(['time', 'posicao'])['probabilidade'].rank()
%timeit df['vencedor_probabilidade_rank'] = df.groupby(['time', 'posicao', 'vencedor'])['probabilidade'].rank()

557 ms ± 200 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
700 ms ± 233 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
809 ms ± 335 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
CPU times: total: 13.6 s
Wall time: 17.5 s


## Aumentando os dados para teste final

In [45]:
df = setup_dataframe(10_000_000)
%timeit df['idade_rank'] = df.groupby(['time', 'posicao'])['idade'].rank()
%timeit df['probabilidade_rank'] = df.groupby(['time', 'posicao'])['probabilidade'].rank()
%timeit df['vencedor_probabilidade_rank'] = df.groupby(['time', 'posicao', 'vencedor'])['probabilidade'].rank()

7.74 s ± 1.08 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
9.71 s ± 718 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
11.3 s ± 1.23 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [46]:
df = setup_dataframe(10_000_000)
df = alterando_dtypes(df)
%timeit df['idade_rank'] = df.groupby(['time', 'posicao'])['idade'].rank()
%timeit df['probabilidade_rank'] = df.groupby(['time', 'posicao'])['probabilidade'].rank()
%timeit df['vencedor_probabilidade_rank'] = df.groupby(['time', 'posicao', 'vencedor'])['probabilidade'].rank()

5.53 s ± 330 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
8.45 s ± 1.24 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
8.42 s ± 474 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Fontes:
- [Rob Mulla](https://www.youtube.com/@robmulla/about)
- [analyticsvidhya](https://www.analyticsvidhya.com/blog/2021/04/how-to-reduce-memory-usage-in-python-pandas/)